In [1]:
import ee
import geemap

ee.Authenticate()
ee.Initialize()

In [2]:
import sys
import ee


ACOLITE_PATH = r'C:\Users\sergi\Documents\repos\gee_acolite\src'

sys.path.append(ACOLITE_PATH)

import ACOLITE

In [3]:
bounds = [-48.490219, -27.634741, -48.423615, -27.509432]
# start, end = '2018-01-30', '2018-01-31'
start, end = '2017-01-12', '2017-01-13'
tile = '22JGQ'

roi = ee.Geometry.Rectangle(bounds)

sentinel2_l1 = ACOLITE.search_with_clouds(roi, start, end, tile = tile)
print(f'Tile {tile}, L1 to L2 via Fixed DSF AC')
print(f'Collection size {sentinel2_l1.size().getInfo()}')

sentinel2_l1r_rrs = sentinel2_l1.map(ACOLITE.filter_bands).map(ACOLITE.to_rrs)
sentinel2_list = sentinel2_l1r_rrs.toList(sentinel2_l1r_rrs.size())

sentinel2_l2r_rrs = ee.ImageCollection.fromImages( [ ACOLITE.dsf( ee.Image(sentinel2_list.get(i)) )[0] for i in range(sentinel2_l1r_rrs.size().getInfo()) ] )
sentinel2_l2r_rrs_deglint = ee.ImageCollection.fromImages( [ ACOLITE.remove_glint(*ACOLITE.dsf( ee.Image(sentinel2_list.get(i)))) for i in range(sentinel2_l1r_rrs.size().getInfo()) ] )

Tile 22JGQ, L1 to L2 via Fixed DSF AC
Collection size 1


In [4]:
Map = geemap.Map()
Map.add_layer_control()

vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 0.5,
    'gamma': 1.4
}

Map.addLayer(sentinel2_l1r_rrs.first(), vis_params, "Sentinel-2 L1 RGB")
Map.addLayer(sentinel2_l2r_rrs.first(), vis_params, "Sentinel-2 L2 RGB")
Map.addLayer(sentinel2_l2r_rrs_deglint.first(), vis_params, "Sentinel-2 L2 Deglint RGB")

Map.centerObject(roi)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…